In [12]:
from mujoco_py import load_model_from_path, MjSim, MjViewer
from mujoco_py import MjRenderContextOffscreen
import mujoco_py

from IPython.display import HTML, display
from matplotlib import pyplot as plt
from copy import deepcopy
import numpy as np
import imageio
import base64
import types
import glob
import cv2
import io

In [13]:
def reset_mocap_welds(sim):
    """Resets the mocap welds that we use for actuation.
    """
    if sim.model.nmocap > 0 and sim.model.eq_data is not None:
        for i in range(sim.model.eq_data.shape[0]):
            if sim.model.eq_type[i] == mujoco_py.const.EQ_WELD:
                sim.model.eq_data[i, :] = np.array(
                    [0., 0., 0., 1., 0., 0., 0.])
    sim.forward()

In [2]:
def save_video(frames, filename='video/mujoco.mp4', fps=60):
    writer = imageio.get_writer(filename, fps=fps)
    for f in frames:
        writer.append_data(f)
    writer.close()
    
def show_video(filname='video/mujoco.mp4'):
    mp4list = glob.glob(filname)
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
        
def show_image(img):
    #cv2.imshow("test", img)
    plt.figure(figsize = (16,9))
    plt.axis('off')
    plt.imshow(img)

In [3]:
#model = load_model_from_path('UR5_w_robotiq.xml')
model = load_model_from_path('fetch_robot/fetch/pick_and_place.xml')
#sim = MjSim(model)
n_substeps = 20
sim = MjSim(model, nsubsteps=n_substeps)
viewer = MjViewer(sim)

Creating window glfw


In [4]:
sim.nsubsteps

20

In [5]:
num_action = len(sim.data.ctrl[:])
print(num_action)

2


In [6]:
sim.model.joint_names

('robot0:slide0',
 'robot0:slide1',
 'robot0:slide2',
 'robot0:torso_lift_joint',
 'robot0:head_pan_joint',
 'robot0:head_tilt_joint',
 'robot0:shoulder_pan_joint',
 'robot0:shoulder_lift_joint',
 'robot0:upperarm_roll_joint',
 'robot0:elbow_flex_joint',
 'robot0:forearm_roll_joint',
 'robot0:wrist_flex_joint',
 'robot0:wrist_roll_joint',
 'robot0:r_gripper_finger_joint',
 'robot0:l_gripper_finger_joint',
 'object0:joint')

In [7]:
arm_joint_list = ['torso_lift_joint', 'head_pan_joint', 'head_tilt_joint', 'shoulder_pan_joint', 'shoulder_lift_joint',\
                  'upperarm_roll_joint', 'elbow_flex_joint',\
                  'forearm_roll_joint', 'wrist_flex_joint', 'wrist_roll_joint']
arm_joint_list = ['robot0:{}'.format(joint_name) for joint_name in arm_joint_list]
gripper_joint_list = ['r_gripper_finger_joint', 'l_gripper_finger_joint']
gripper_joint_list = ['robot0:{}'.format(joint_name) for joint_name in gripper_joint_list]
str = ''
for joint_name in gripper_joint_list:
    str += "{} : {:.3f}, ".format(joint_name, sim.data.get_joint_qpos(joint_name))
print(str)
#print(sim.data.get_joint_qpos('finger_joint'))
#print(sim.data.get_joint_qpos('left_inner_knuckle_joint'))

robot0:r_gripper_finger_joint : 0.000, robot0:l_gripper_finger_joint : 0.000, 


In [8]:
lookat_refer = sim.data.get_body_xpos('object0')
viewer.cam.lookat[0] = lookat_refer[0]
viewer.cam.lookat[1] = lookat_refer[1]
viewer.cam.lookat[2] = lookat_refer[2]
viewer.cam.azimuth = 150
viewer.cam.elevation = -30
viewer.cam.distance = 1.5

In [9]:
sim.model.body_names

('world',
 'floor0',
 'robot0:mocap',
 'robot0:base_link',
 'robot0:torso_lift_link',
 'robot0:head_pan_link',
 'robot0:head_tilt_link',
 'robot0:head_camera_link',
 'robot0:head_camera_rgb_frame',
 'robot0:head_camera_rgb_optical_frame',
 'robot0:head_camera_depth_frame',
 'robot0:head_camera_depth_optical_frame',
 'robot0:shoulder_pan_link',
 'robot0:shoulder_lift_link',
 'robot0:upperarm_roll_link',
 'robot0:elbow_flex_link',
 'robot0:forearm_roll_link',
 'robot0:wrist_flex_link',
 'robot0:wrist_roll_link',
 'robot0:gripper_link',
 'robot0:gipper_camera_link',
 'robot0:gripper_camera_rgb_frame',
 'robot0:gripper_camera_rgb_optical_frame',
 'robot0:gripper_camera_depth_frame',
 'robot0:gripper_camera_depth_optical_frame',
 'robot0:r_gripper_finger_link',
 'robot0:l_gripper_finger_link',
 'robot0:estop_link',
 'robot0:laser_link',
 'robot0:torso_fixed_link',
 'robot0:external_camera_body_0',
 'table0',
 'object0')

In [10]:
print(sim.data.get_body_xpos('object0'))
print(sim.data.get_body_xpos('robot0:gripper_link'))

[1.23 0.75 0.5 ]
[1.628 0.75  0.786]


In [14]:
reset_mocap_welds(sim)

In [15]:
sim.data.mocap_pos

array([[0., 0., 0.]])

In [12]:
init_arm_pos = np.zeros(len(arm_joint_list))
init_gripper_pos = 0.0
gripper_ctrl = 0.05
for joint_idx, joint_name in enumerate(arm_joint_list):
    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    sim.data.set_joint_qvel(joint_name, 0.0)
for joint_idx, joint_name in enumerate(gripper_joint_list):
    sim.data.set_joint_qpos(joint_name, init_gripper_pos)
    sim.data.set_joint_qvel(joint_name, 0.0)
sim.step()

frames = []
cnt = 0
for j in range(300):
    for joint_idx, joint_name in enumerate(arm_joint_list):
        sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
        sim.data.set_joint_qvel(joint_name, 0.0)

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
    
    sim.step()

    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

init_arm_pos = np.zeros(len(arm_joint_list))
init_arm_pos[4] = np.pi/6
init_arm_pos[6] = -np.pi/3
init_arm_pos[8] = np.pi/1.5
for j in range(100):
    for joint_idx, joint_name in enumerate(arm_joint_list):
        cur_pos = sim.data.get_joint_qpos(joint_name)
        cur_vel = sim.data.get_joint_qvel(joint_name)
        sim.data.set_joint_qpos(joint_name, cur_pos - 0.1*(cur_pos - init_arm_pos[joint_idx]) - 0.001*cur_vel)
        #sim.data.set_joint_qpos(joint_name, cur_pos - 0.8*(cur_pos - init_arm_pos[joint_idx]))
        #sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
        sim.data.set_joint_qvel(joint_name, 0.0)

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
    
    sim.step()
    
    #cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(len(frames)/(400*60)))
show_video('video/mujoco.mp4')

0.016871987618192905
-0.535635444048848
-0.9576375875876155
-1.2778334740294353
-1.5309552183275574
-1.7536631823119238
-1.953495488982547
-2.119215771711253
-2.2661220983526387
-2.401246266156097
-2.527064518555572
-2.644261321670774
-2.753061919085589
-2.8536913634894385
-2.9232985056533387
-2.998346140494117
-3.074449164208212
-3.146719939811241
-3.2137040455737487
-3.2752123297082534
-3.331473839101036
-3.3828396301125796
-3.4296832078727557
-3.4723681428515025
-3.511237985132596
-3.5466135241112084
-3.5787925314571543
-3.6080504052849762
-3.6346411773543346
-3.6587986786312103
-3.680737770502116
-3.700655591370098
-3.7187327878343144
-3.7351347105516455
-3.7500125618887736
-3.7635044873222894
-3.7757366060027553
-3.7868239783877105
-3.7968715106338973
-3.8059747967061237
-3.814220900041306
-3.8216890771997623
-3.828451446315452
-3.8345736033767617
-3.8401151894732517
-3.8451304121608234
-3.8496685240543247
-3.85377426166905
-3.8574882474159407
-3.860847357519119
-3.863885058476815

In [13]:
print(sim.data.get_body_xpos('robot0:gripper_link'))

for gripper_idx in range(len(gripper_joint_list)):
    cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
    print(cur_pos)

[1.62075408 0.75000149 0.7699148 ]
0.050014855484441366
0.05001485638384129


In [12]:
gripper_ctrl = -0.0005
init_arm_pos = np.zeros(len(arm_joint_list))
init_arm_pos[4] = np.pi/6
init_arm_pos[6] = -np.pi/3
init_arm_pos[8] = np.pi/1.5

frames = []
cnt = 0
for j in range(1000):
    for joint_idx, joint_name in enumerate(arm_joint_list):
        sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
        sim.data.set_joint_qvel(joint_name, 0.0)

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[1 - gripper_idx] = cur_pos + gripper_ctrl
    print(sim.data.ctrl)
    
    sim.step()
    
    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(1.0/viewer._time_per_render))
show_video('video/mujoco.mp4')

[0.0495146  0.04951511]
[0.04951401 0.04951454]
[0.04951286 0.04951343]
[0.04951116 0.04951177]
[0.04950892 0.04950959]
[0.04950616 0.04950687]
[0.04950288 0.04950365]
[0.04949909 0.04949993]
[0.04949481 0.04949571]
[0.04949004 0.049491  ]
[0.04948479 0.04948583]
[0.04947908 0.04948018]
[0.0494729  0.04947408]
[0.04946627 0.04946753]
[0.04945919 0.04946054]
[0.04945169 0.04945312]
[0.04944375 0.04944527]
[0.0494354  0.04943701]
[0.04942663 0.04942834]
[0.04941746 0.04941927]
[0.0494079  0.04940981]
[0.04939795 0.04939996]
[0.04938762 0.04938974]
[0.04937692 0.04937914]
[0.04936584 0.04936818]
[0.04935441 0.04935686]
[0.04934263 0.0493452 ]
[0.0493305  0.04933319]
[0.04931803 0.04932084]
[0.04930523 0.04930816]
[0.0492921  0.04929516]
[0.04927865 0.04928184]
[0.04926489 0.0492682 ]
[0.04925082 0.04925426]
[0.04923644 0.04924002]
[0.04922177 0.04922549]
[0.0492068  0.04921066]
[0.04919155 0.04919555]
[0.04917602 0.04918016]
[0.04916022 0.0491645 ]
[0.04914414 0.04914857]
[0.0491278  0.04

[0.04077422 0.04084958]
[0.04074413 0.04081975]
[0.04071404 0.04078993]
[0.04068396 0.0407601 ]
[0.04065387 0.04073027]
[0.04062378 0.04070044]
[0.04059369 0.04067061]
[0.0405636  0.04064078]
[0.04053351 0.04061095]
[0.04050341 0.04058111]
[0.04047332 0.04055128]
[0.04044323 0.04052145]
[0.04041313 0.04049161]
[0.04038304 0.04046178]
[0.04035294 0.04043194]
[0.04032284 0.0404021 ]
[0.04029274 0.04037227]
[0.04026265 0.04034243]
[0.04023255 0.04031259]
[0.04020245 0.04028275]
[0.04017235 0.04025291]
[0.04014225 0.04022307]
[0.04011215 0.04019323]
[0.04008204 0.04016339]
[0.04005194 0.04013355]
[0.04002184 0.04010371]
[0.03999174 0.04007387]
[0.03996163 0.04004402]
[0.03993153 0.04001418]
[0.03990142 0.03998434]
[0.03987132 0.03995449]
[0.03984121 0.03992465]
[0.0398111 0.0398948]
[0.039781   0.03986496]
[0.03975089 0.03983511]
[0.03972078 0.03980527]
[0.03969067 0.03977542]
[0.03966056 0.03974557]
[0.03963045 0.03971573]
[0.03960034 0.03968588]
[0.03957023 0.03965603]
[0.03954012 0.0396

[0.03022914 0.03039928]
[0.030199   0.03036942]
[0.03016886 0.03033956]
[0.03013871 0.0303097 ]
[0.03010857 0.03027984]
[0.03007843 0.03024998]
[0.03004829 0.03022012]
[0.03001815 0.03019027]
[0.02998801 0.03016041]
[0.02995787 0.03013055]
[0.02992772 0.03010069]
[0.02989758 0.03007083]
[0.02986744 0.03004097]
[0.0298373  0.03001112]
[0.02980716 0.02998126]
[0.02977702 0.0299514 ]
[0.02974687 0.02992154]
[0.02971673 0.02989168]
[0.02968659 0.02986182]
[0.02965645 0.02983197]
[0.02962631 0.02980211]
[0.02959617 0.02977225]
[0.02956602 0.02974239]
[0.02953588 0.02971253]
[0.02950574 0.02968268]
[0.0294756  0.02965282]
[0.02944546 0.02962296]
[0.02941531 0.0295931 ]
[0.02938517 0.02956324]
[0.02935503 0.02953339]
[0.02932489 0.02950353]
[0.02929475 0.02947367]
[0.0292646  0.02944381]
[0.02923446 0.02941396]
[0.02920432 0.0293841 ]
[0.02917418 0.02935424]
[0.02914403 0.02932438]
[0.02911389 0.02929453]
[0.02908375 0.02926467]
[0.02905361 0.02923481]
[0.02902346 0.02920495]
[0.02899332 0.02

In [13]:
'''
init_arm_pos = np.zeros(len(arm_joint_list))
init_arm_pos[4] = np.pi/6
init_arm_pos[6] = -np.pi/3
init_arm_pos[8] = np.pi/1.5

frames = []
cnt = 0
for j in range(300):
    for joint_idx, joint_name in enumerate(arm_joint_list):
        if joint_idx == 8:
            sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx] - 0.5*j/300)
        else:
            sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
        sim.data.set_joint_qvel(joint_name, 0.0)
    sim.data.ctrl[0] = gripper_ctrl
    sim.data.ctrl[1] = gripper_ctrl
    sim.step()
    
    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

save_video(frames, 'video/mujoco.mp4', int(0.5/viewer._time_per_render))
show_video('video/mujoco.mp4')
'''

"\ninit_arm_pos = np.zeros(len(arm_joint_list))\ninit_arm_pos[4] = np.pi/6\ninit_arm_pos[6] = -np.pi/3\ninit_arm_pos[8] = np.pi/1.5\n\nframes = []\ncnt = 0\nfor j in range(300):\n    for joint_idx, joint_name in enumerate(arm_joint_list):\n        if joint_idx == 8:\n            sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx] - 0.5*j/300)\n        else:\n            sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])\n        sim.data.set_joint_qvel(joint_name, 0.0)\n    sim.data.ctrl[0] = gripper_ctrl\n    sim.data.ctrl[1] = gripper_ctrl\n    sim.step()\n    \n    cnt += sim.model.opt.timestep*sim.nsubsteps/viewer._time_per_render\n    while cnt > 0:\n        cnt -= 1\n        img = viewer._read_pixels_as_in_window()\n        frames.append(img)\n\nsave_video(frames, 'video/mujoco.mp4', int(0.5/viewer._time_per_render))\nshow_video('video/mujoco.mp4')\n"